# Comparing meta-learners with different ensembles of trees as base learners

## Introduction

Meta-learners are a class of models used for estimating *heterogeneous causal effect* of a certain treatement. Here by heterogeneity, we mean that the effect of the treatement may vary across individuals. They approach this problem by decomposing it to several subregressions. The models used for these subregressions are called *base learners*. Some of the most popular base learners are tree based ensembles, in particular Random Forest and Bayesian Additive Regression Trees (BART) due to their flexibility, however any machine learning algorithm suitable for regression problems (like generalized linear regressions, gaussian processes, etc.) can be used.

We will denote by $Y$ a $d$-dimensional outcome vector, by $X \in \mathbb{R}^{d \times n}$ a feature matrix containing potential confounders, by $W \in \{0, 1\}^d$ the treatment assignment indicator and by  $Y(0) \in \mathbb{R}^d$ and $Y(1) \in \mathbb{R}^d$ the potential outcome of corresponding units when assigned to the control and the treatement group respectively. Note that in general, we only observe one of the potential outcomes, furthermore
$$ Y = W Y(1) + (1 - W) Y(0).$$

Our task is to estimate the *conditional average treatement effect (CATE)*, defined as 
$$\tau(x) = \mathbb{E}\left[Y(1) - Y(0) \mid X=x\right].$$

## Data generation

In this notebook we will deal with synthetic data, so that we can quantify exactly the mean squared error of our CATE estimations. The data generating is of the form
$$y = \operatorname{sinc}(\alpha X) + W \sigma(\beta X) + \varepsilon,$$
where $\varepsilon_i \sim N(0, 1)$ and $W_i \sim \operatorname{Bernoulli}\left(\frac14\right)$ are i.i.d., $$\sigma(x) = \frac{e^x}{1 + e^x}, \quad \text{and} \quad \operatorname{sinc}(x) = \frac{\sin(x)}{x}$$ are applied coordinate-wise.

Here, CATE can be expressed as $$\tau(X) = \sigma(\beta X).$$

In [21]:
import numpy as np
import pandas as pd
from causalpy.skl_meta_learners import SLearner, TLearner, XLearner, DRLearner
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
import seaborn as sns
import matplotlib.pyplot as plt


np.random.seed(42)

def sigmoid(x):
    return np.exp(x) / (1 + np.exp(x))

def sinc(x):
    return np.where(x==0, 1, np.sin(x) / x)

def summarize_learner(learner):
    learner.summary(n_iter=100)
    print(f"Actual average treatement effect: {treatment_effect.mean()}")
    print(f"MSE: {mean_squared_error(treatment_effect, learner.cate)}")


def create_synthetic_data(*shape):
    X = np.random.rand(*shape)

    α = np.random.rand(shape[1])
    β = np.random.rand(shape[1])

    treatment = np.random.binomial(n=1, p=.25, size=shape[0])
    treatment_effect = sigmoid(X @ β)

    noise = np.random.rand(shape[0])

    y = sinc(X @ α) + treatment_effect * treatment + noise

    X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(shape[1])])
    y = pd.Series(y, name='target')
    treated = pd.Series(treatment, name='treatment')
    return X, y, treated, treatment_effect

X, y, treated, treatment_effect = create_synthetic_data(500, 20)

## S-learner

The simplest meta-learner is the *S-learner*, which predicts the potential outcome vectors by treating the treatement assignment indicator as a feature and estimates

$$\mu(x, w) := \mathbb{E}[Y \mid X=x, W=w]$$

with an appropriate base learner. The "S" in S-learner stands for "single", as it uses a single model to estimate CATE. We will denote this estimation with $\hat{\mu}(x, w)$. Then the S-learner approximates CATE as  

$$\hat{\tau_S}(x) = \hat{\mu}(x, 1) - \hat{\mu}(x, 0).$$

In the following example we will choose `HistGradientBoostingRegressor` as base learner.

In [ ]:
%%time
# Utility for printing some statistics.
slearner = SLearner(
    X=X,
    y=y,
    treated=treated,
    model=HistGradientBoostingRegressor()
)

summarize_learner(slearner)

## T-learner

The *T-learner* is a closely related meta-learner. Here, instead of including the treatment assignment indicator as a feature, we estimate the potential outcomes in two different regressions. Namely, we estimate
$$\mu_{\operatorname{treated}}(x) := \mathbb{E}[Y(1) \mid X=x], \quad \text{and} \quad \mu_{\operatorname{untreated}}(x) := \mathbb{E}[Y(0) \mid X=x]$$
separately and define estimate the cate as
$$\hat{\tau}_T(x) = \hat{\mu}_{\operatorname{treated}}(x) - \hat{\mu}_{\operatorname{untreated}}(x),$$
where $\hat{\mu}_{\operatorname{treated}}$ and $\hat{\mu}_{\operatorname{untreated}}$ are the estimated functions. 

If we choose our base-learners to be a forest based ensemble, in the case of the S-learner, during the fitting process splits on the treatement assignment indicator can happen *anywhere* in the trees. In the fitting process of the T-learner however, we essentially force a split at the first level. This difference means that the S-learner is more flexible, but especially when using regularization, it is prone to disregard the effect of the treatment. On the other hand the T-learner uses two separate models that do not share information between each other, which may lead to overfitting if one of the treatement groups is small.

In [ ]:
%%time
tlearner = TLearner(
    X=X,
    y=y,
    treated=treated,
    model=HistGradientBoostingRegressor()
)

summarize_learner(tlearner)

# X-learner
Similarly, to the T-learner, the X-learner first estimates $\mu_{\operatorname{treated}}$ and $\mu_{\operatorname{untreated}}$. Next, it computes 

$$D(1) := Y(1) - \hat{\mu}_{\operatorname{untreated}}(X(1)), \quad \text{and} \quad D(0) = \hat{\mu}_{\operatorname{treated}}(X(0)) - Y(0).$$

Note that if $\hat{\mu}_{\operatorname{untreated}} = \mu_{\operatorname{untreated}}$ and $\hat{\mu}_{\operatorname{treated}} = \mu_{\operatorname{treated}}$, then $\tau(x) = \mathbb{E}[D(1) | X=x] = \mathbb{E}[D(0) | X=x]$, so CATE may be estimated by regressing on $D(0)$ or $D(1)$. Let us denote the estimations obtained this way by $\tau_0$ and $\tau_1$, respectively. 

Finally, the X-learner computes a weighted average of the two CATE estimates, that is 
$$\hat{\tau}_X(x) = g(x)\hat{\tau}_0 + (1 - g(x))\hat{\tau_1}(x),$$
for some weight function $g: \mathbb{R}^d \rightarrow [0, 1]$.

In CausalPy, we obtain $g$ as an estimation of the so-called *propensity score*, that is the conditional probability of a unit receiving treatment given $X$. By default, we use logistic regression to obtain this estimation. However, in case one of the treatement groups is small, it makes sense to set $g \equiv 0$ or $g \equiv 1$.

In [ ]:
%%time
# Using logistic regression based propensity score
xlearner = XLearner(
    X=X,
    y=y,
    treated=treated,
    model=HistGradientBoostingRegressor()
)

summarize_learner(xlearner)

In [ ]:
%%time
# Using constant g
xlearner_mf = XLearner(
    X=X,
    y=y,
    treated=treated,
    model=HistGradientBoostingRegressor(),
    propensity_score_model=DummyClassifier(strategy="most_frequent")
)

summarize_learner(xlearner_mf)

# DR-learner
The *doubly robust learner*, also known as the *DR-learner*, is the last CATE estimator that we are going to look at.

In [ ]:
%%time
drlearner = DRLearner(
    X=X,
    y=y,
    treated=treated,
    model=HistGradientBoostingRegressor(),
    propensity_score_model=XGBClassifier()
)

summarize_learner(drlearner)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, AdaBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

def compute_mse(learner, m):
    l = learner(X, y, treated, m)
    return mean_squared_error(treatment_effect, l.cate)

models = [
    ("Random forest", RandomForestRegressor()),
    ("Hist gradient boosting", HistGradientBoostingRegressor()),
    ("AdaBoost", AdaBoostRegressor()),
    ("XGBoost", XGBRegressor()),
    ("LightGBM", LGBMRegressor()),
    ("SVR", SVR())
]

learners = [
    ("S-learner", SLearner),
    ("T-learner", TLearner),
    ("X-learner", XLearner),
    ("DR-learner", DRLearner)
]
bench = pd.DataFrame({
    m_name: {
        learner_name:  compute_emse(learner, m) for learner_name, learner in learners
    } for m_name, m in models
})

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(bench, annot=True, ax=ax)